# Classification

Elements of Data Science

by [Allen Downey](https://allendowney.com)

[MIT License](https://opensource.org/licenses/MIT)

### Goals

The primary goal of this notebook is to review ways to evaluate binary classification algorithms.  We'll start with the confusion matrix and the metrics that are derived from it, including accuracy, sensitivity, specificity, predictive value, false positive rate, and false negative rate.

Then...

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Machine Bias

As an example of a classification problem, I will replicate the analysis reported in
"[Machine Bias](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing)", by Julia Angwin, Jeff Larson, Surya Mattu and Lauren Kirchner, and published by [ProPublica](https://www.propublica.org), in May 2016.

This article is about a statistical tool called COMPAS which was used by judges in Broward County, Florida, to inform their sentencing decisions, specifically which convicted defendents would be put in prison and which would be released on parole.

COMPAS uses information about the defendants to generate a "risk score" which is intended to quantify the risk that defendant would commit another crime is released.

The authors of the ProPublica article used public data to assess the accuracy of those risk scores.  They explain:

> We obtained the risk scores assigned to more than 7,000 people arrested in Broward County, Florida, in 2013 and 2014 and checked to see how many were charged with new crimes over the next two years, the same benchmark used by the creators of the algorithm.

[In their notebook](https://github.com/propublica/compas-analysis/blob/master/Compas%20Analysis.ipynb), they explain in more detail:

> We filtered the underlying data from Broward county to include only those rows representing people who had either recidivated in two years, or had at least two years outside of a correctional facility.
>
> [...]
>
> Next, we sought to determine if a person had been charged with a new crime subsequent to crime for which they were COMPAS screened. We did not count traffic tickets and some municipal ordinance violations as recidivism. We did not count as recidivists people who were arrested for failing to appear at their court hearings, or people who were later charged with a crime that occurred prior to their COMPAS screening.

They use this data to evaluate how well COMPAS predicts the risk that a defendant will be charged with another crime within two years of their release.

Among their findings, they report:

> ... the algorithm was somewhat more accurate than a coin flip. Of those deemed likely to re-offend, 61 percent were arrested for any subsequent crimes within two years.
> 
> ... In forecasting who would re-offend, the algorithm made mistakes with black and white defendants at roughly the same rate but in very different ways.
>
> * The formula was particularly likely to falsely flag black defendants as future criminals, wrongly labeling them this way at almost twice the rate as white defendants.
>
> * White defendants were mislabeled as low risk more often than black defendants.

I'll start by replicating their analysis, and use this example to explain the metrics used to assess binary classification algorithms.

## Data

Thank you to ProPublica and the authors of "Machine Bias" for making their data and analysis freely available.  They are a model of open science.

[This repository](https://github.com/propublica/compas-analysis) the data and analysis pipeline described on [this web page](https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm).

The terms of use for the data [are here](https://www.propublica.org/datastore/terms).  In compliance with those terms, I am not redistributing the data.

The following cell downloads the data file we'll use directly from their repository.

In [2]:
import os

if not os.path.exists('compas-scores-two-years.csv'):
    !wget https://github.com/propublica/compas-analysis/raw/master/compas-scores-two-years.csv

The following cells read the data file and display the column headings.  I have not found documentation for the columns in this dataset, so we'll have to infer what they mean based on the column names and how they are used in the original analysis.

In [3]:
cp = pd.read_csv("compas-scores-two-years.csv")
cp.shape

(7214, 53)

The dataset includes 7214 rows, one for each defendant, and 53 columns.

In [4]:
cp.columns

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')

Here are the libraries and functions I'll use for my analysis.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
def decorate(**options):
    """Decorate the current axes.
    
    Call decorate with keyword arguments like
    decorate(title='Title',
             xlabel='x',
             ylabel='y')
             
    The keyword arguments can be any of the axis properties
    https://matplotlib.org/api/axes_api.html
    """
    plt.gca().set(**options)
    plt.tight_layout()

## The Confusion Matrix

The authors of "Machine Bias" describe their analysis in more detail in [this article](https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm).  It includes this table, which summarizes many of the results they report:

<img src='https://raw.githubusercontent.com/AllenDowney/ElementsOfDataScience/master/machine_bias_table.png'>


The table summarizes results for all defendants and two subgroups: defendants classified as white ("Caucasian" in the original dataset) and black ("African-American").

For each group, the summary includes a [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) and a number of metrics, including

* FP rate: false positive rate
* FN rate: false negative rate
* PPV: positive predictive value
* NPV: negative predictive value
* LR+: positive likelihood ratio 
* LR-: negative likelihood ratio 

I will explain what these metrics mean and how to compute them, and we'll replicate the results in this table.

But first let's examine the data and compute a couple of recoded variables.

The following function displays the values in a Series and the number of times each value appears.

In [7]:
def values(series):
    """Count the values and sort.
    
    series: pd.Series
    
    returns: series mapping from values to frequencies
    """
    return series.value_counts(dropna=False).sort_index()

Here are the values of `decile_score` which is the output of the COMPAS algorithm.  `1` is the lowest risk category; `10` is the highest.

In [8]:
values(cp['decile_score'])

1     1440
2      941
3      747
4      769
5      681
6      641
7      592
8      512
9      508
10     383
Name: decile_score, dtype: int64

It's important to note that COMPAS is not a binary classifier; that is, it does no predict that a defendant will or will not recidivate.  Rather, it gives each defendant a score that is intended to reflect the risk that they will recidivate.

In order to evaluate the performance of COMPAS, the authors of the ProPublica article chose a threshold, `4`, and define decile scores at or below the threshold to be "low risk", and scores above the threshold to be "high risk".

The choice of the threshold is arbitrary.  Later, we will see what happens with other choice, but we'll start by replicating the original analysis.

`high_risk` is a Boolean Series that's `True` for respondents with a decile score greater than 4.

In [9]:
high_risk = (cp['decile_score'] > 4)
high_risk.name = 'HighRisk'
values(high_risk)

False    3897
True     3317
Name: HighRisk, dtype: int64

The column `two_year_recid` indicates whether a defendant was charged with another crime during a two year period, after the original charge, when they were not in a correctional facility.

In [10]:
values(cp['two_year_recid'])

0    3963
1    3251
Name: two_year_recid, dtype: int64

`new_charge_2` is a Boolean Series that is `True` for defendants who were charged with another crime within two years.

In [11]:
new_charge_2 = (cp['two_year_recid'] == 1)
new_charge_2.name = 'NewCharge2'
values(new_charge_2)

False    3963
True     3251
Name: NewCharge2, dtype: int64

If we make a cross-tabulation of `new_charge_2` and `high_risk`, the result is a DataFrame that indicates how many defendants are in each of four groups:

In [12]:
pd.crosstab(new_charge_2, high_risk)

HighRisk,False,True
NewCharge2,,
False,2681,1282
True,1216,2035


This table is called a [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) or error matrix.  Reading from left to right and top to bottom, the elements of the matrix show the number of respondents who were:

* Classified as low risk and not arrested again: there were 2681 *true negatives*; that is, cases where the test is negative (not high risk) and the prediction turned out to be true (not arrested again).

* High risk and not arrested: there were 1282 *false positives*, cases where the test is positive and the prediction was false.

* Low risk and arrested: there were 1216 *false negatives*, cases where the test is negative (low risk) and the prediction is false (the defendant was arrested again).

* High risk and arrested: there were 2035 *true positives*, cases where the test is positive and the prediction was true.

The values in this matrix are consistent with the values in the ProPublica article, so we can confirm that we are reading the data and replicating their analysis correctly.

In the next section we'll see several ways to use the confusion matrix to quantify the accuracy of the test.

But first let's compute confusion matrices for white and black defendants.

Here are the values of `race`:

In [13]:
values(cp['race'])

African-American    3696
Asian                 32
Caucasian           2454
Hispanic             637
Native American       18
Other                377
Name: race, dtype: int64

Here's a Boolean Series that's true for white defendants.

In [14]:
white = (cp['race'] == 'Caucasian')
white.name = 'white'
values(white)

False    4760
True     2454
Name: white, dtype: int64

And here's the confusion matrix for white defendants.

In [15]:
pd.crosstab(new_charge_2[white], high_risk[white])

HighRisk,False,True
NewCharge2,,
False,1139,349
True,461,505


Again, these results are consistent with the table in the ProPublica article.

`black` is a Boolean Series that is `True` for black defendants.

In [16]:
black = (cp['race'] == 'African-American')
black.name = 'black'
values(black)

False    3518
True     3696
Name: black, dtype: int64

And here is the confusion matrix for black defendants.

In [17]:
pd.crosstab(new_charge_2[black], high_risk[black])

HighRisk,False,True
NewCharge2,,
False,990,805
True,532,1369


Again these results are consistent with the ProPublica article.

## Arranging the confusion matrix

In the previous section I arranged the confusion matrix to be consistent with the ProPublica article, to make it easy to check for consistency.

In general, there are many possible ways to arrange a confusion matrix: you can put the predictions along the columns and the actual conditions along the rows, or the other way around, and you can sort the rows and columns in either order.

There is no universal standard arrangement, but the one that seems to be most common is the one on the [confusion matrix Wikipedia page](https://en.wikipedia.org/wiki/Confusion_matrix), which looks like this:

<img width=400, src='https://raw.githubusercontent.com/AllenDowney/ElementsOfDataScience/master/confusion_matrix1.png'>

In this arrangement:

* The predictions are along the rows.

* The actual results are along the columns.

* The rows and columns are sorted so true positives are in the upper left; true negatives are in the lower right.

In the context of the ProPublica article:

* "Predicted condition positive" means the defendant is classified as high risk

* "Predicted condition negative" means low risk.

* "Condition positive" means the defendant was arrested again (recidivated).

* "Condition negative" means they were not arrested (survived).

The following function make a confusion matrix with this arrangement:

In [18]:
def make_matrix(cp, subgroup, threshold=4):
    """Make a confusion matrix.
    
    cp: DataFrame
    subgroup: Boolean Series
    
    returns: DataFrame containing the confusion matrix
    """
    a = np.where(cp['decile_score'] > threshold, 
                 'Positive', 
                 'Negative')
    high_risk = pd.Series(a, name='Predicted')
    
    a = np.where(cp['two_year_recid'] == 1, 
                 'Condition', 
                 'No Condition')
    new_charge_2 = pd.Series(a, name='Actual')

    matrix = pd.crosstab(high_risk[subgroup], new_charge_2[subgroup])
    matrix.sort_index(axis=0, ascending=False, inplace=True)
    
    return matrix

Here's the confusion matrix for white defendants.

In [19]:
matrix_white = make_matrix(cp, white)
matrix_white

Actual,Condition,No Condition
Predicted,,
Positive,505,349
Negative,461,1139


In [20]:
matrix_black = make_matrix(cp, black)
matrix_black

Actual,Condition,No Condition
Predicted,,
Positive,1369,805
Negative,532,990


In [21]:
all_defendants = np.ones_like(white)
matrix_all = make_matrix(cp, all_defendants)
matrix_all

Actual,Condition,No Condition
Predicted,,
Positive,2035,1282
Negative,1216,2681


## Metrics

Based on these results, how accurate is COMPAS as a binary classifier?  Well, it turns out that there are a *lot* of ways to answer that question.

One of the simple ones is overall **accuracy** which is the fraction (or percentage) of correct predictions.

To compute accuracy, it is convenient to extract from the confusion matrix the number of true negatives, false positives, false negatives, and true positives.

In [22]:
tp, fp, fn, tn = matrix_all.to_numpy().flatten()

The number of true predictions is `tp + tn`.

The number of false predictions is `fp + fn`.

So we can compute the fraction of true predictions like this:

In [23]:
def percent(x, y):
    """Compute the percentage `x/(x+y)*100`.
    """
    return x / (x+y) * 100

In [24]:
accuracy = percent(tp + tn, fp + fn)
accuracy

65.37288605489326

As a way to evaluate a binary classifier, accuracy does not distinguish between true positives and true negatives, or false positives and false negatives.

But it is often important to make these distinctions, because the benefits of true predictions and true negatives might be different, and the costs of false positives and false negatives might be different.

### Predictive value

One way to make these distinctions is to compute the "predictive value" of positive and negative tests. 

* **Positive predictive value (PPV)**, is the fraction of positive tests that are correct.

* **Negative predictive value (NPV)**, is the fraction of negative tests that are correct.

In this example, PPV is the fraction of high risk defendants who were arrested again.  NPV is the fraction of low risk defendants who were not arrested.

The following function takes a confusion matrix and computes these metrics.

In [25]:
def predictive_value(m):
    """Compute positive and negative predictive value.
    
    m: confusion matrix
    """
    tp, fp, fn, tn = m.to_numpy().flatten()
    ppv = percent(tp, fp)
    npv = percent(tn, fn)
    return ppv, npv

Compute predictive value for all defendants.

In [26]:
ppv, npv = predictive_value(matrix_all)
ppv, npv

(61.350618028338864, 68.79651013600206)

Among all defendants, a negative test result is correct about 69% of the time; a positive test is correct about 61% of the time.

## Sensitivity and specificity

Another way to characterize the accuracy of a test is to compute 

* **Sensitivity**, which is the probability of predicting correctly when the condition is present, and 

* **Specificity**, which is the probability of predicting correctly when the condition is absent.

A test is "sensitive" if it correctly detects the positive condition. 
In this example, sensitivity is the fraction of recidivists who were classified as high risk.

And a test is "specific" if it correctly identifies the negative condition.  In this example, specificity is the fraction of non-recidivists who we classified as low risk.

The following function takes a confusion matrix and computes sensitivity and specificity.

In [27]:
def sens_spec(m):
    """Compute sensitivity and specificity.
    
    m: confusion matrix
    """
    tp, fp, fn, tn = m.to_numpy().flatten()
    sens = percent(tp, fn)
    spec = percent(tn, fp)
    return sens, spec

Here are sensitivity and specificity for all defendants.

In [28]:
sens, spec = sens_spec(matrix_all)
sens, spec

(62.59612426945556, 67.65076961897553)

If we evaluate COMPAS as a binary classifier, it is a little more specific than sensitive:

* About 64% of the recidivists were classified as high risk.

* About 67% of the non-recidivists were classified as low risk.

It can be hard to keep all of these metrics straight, especially when you are learning about them for the first time.  This following diagram from Wikipedia might help:

<img width=800, src='https://raw.githubusercontent.com/AllenDowney/ElementsOfDataScience/master/confusion_matrix2.png'>

PPV and sensitivity are similar in the sense that they both have true positives in the numerator.  The difference is the denominator:

* PPV is the ratio of true positives to all positive tests.  So it answers the question, "Of all positive tests, how many are correct?"

* Sensitivity is the ratio of true positives to all positive conditions.  So it answers the question "Of all positive conditions, how many are classified correctly?"

Similary, NPV and sensitivity both have true negatives in the numerator, but:

* NPV is the ratio of true negatives to all negative tests.  It answers, "Of all negative tests, how many are correct?"

* Specificiy is the ratio of true negatives to all negative conditions.  It answers, "Of all negative conditions, how many are classified correctly?"

### False positive and negative rates

The ProPublica article reports PPV and NPV, but instead of sensitivity and specificity, it reports:

* **False positive rate**, which is the ratio of false positives to all negative conditions.  It answers, "Of all negative conditions, how many are misclassified?" 

* **False negative rate**, which is the ratio of false negatives to all positive conditions.  It answers, "Of all positive conditions, how many are misclassified?"

In this example:

* The false positive rate is the fraction of non-recidivists who were classified as high risk.

* The false negative rate is the fraction of recidivists who were classified as low risk.

The following function takes a confusion matrix and computes false positive and false negative rates.

In [29]:
def error_rates(m):
    """Compute false positive and false negative rate.
    
    m: confusion matrix
    """
    tp, fp, fn, tn = m.to_numpy().flatten()
    fpr = percent(fp, tn)
    fnr = percent(fn, tp)
    return fpr, fnr

Here are the error rates for all defendants.

In [30]:
fpr, fnr = error_rates(matrix_all)
fpr, fnr

(32.349230381024476, 37.40387573054445)

False positive rate is the complement of specificity, which means they have to add up to 100%

And false negative rate is the complement of sensitivity.

In [31]:
fpr + spec, fnr + sens

(100.0, 100.0)

So FPR and FNR are just another way of reporting sensitivity and specificity.

In general, I think it is better to report sensitivity and specificity, because

* I think the positive framing is easier to interpret than the negative framing, and

* I find it easier to remember what "sensitivity" and "specificity" mean.

I think "false positive rate" and "false negative rate" are more problematic.  For example, "false positive rate" could just as easily mean either

1. The fraction of positive tests that are incorrect, or

2. The fraction of negative conditions that are misclassified.

It is only a convention that the first is called the "false discovery rate" and the second is the "false positive rate".  I suspect I am not the only one that gets them confused.

So, here's my recommendation: if you have the choice, generally use PPV, NPV, sensitivity and specificity, and avoid the other metrics.

However, since the ProPublica article uses FPR and FNR, I will too.

They also report LR+ and LR-, but those are just combinations of other metrics and not relevant to the current discussion, so I will ignore them.

However, there is one other metric that I think is relevant, and not included in the ProPublica tables: prevalence, which is the fraction of all cases where the condition is positive.  In the example, it's the fraction of defendants who recidivate.

The following function computes prevalence:

In [32]:
def prevalence(df):
    """Compute prevalence.
    
    m: confusion matrix
    """
    tp, fp, fn, tn = df.to_numpy().flatten()
    prevalence = percent(tp+fn, tn+fp)
    return prevalence

And here's the prevalence for all defendants:

In [33]:
prev = prevalence(matrix_all)
prev

45.06515109509287

About 45% of the defendants in this dataset were charged with another crime within two years of their release.

## All metrics

The following function takes a confusion matrix and computes the metrics from the 

In [34]:
def compute_metrics(m, name=''):
    """Compute all metrics.
    
    m: confusion matrix
    
    returns: DataFrame
    """
    fpr, fnr = error_rates(m)
    ppv, npv = predictive_value(m)
    prev = prevalence(m)
    
    index = ['FP rate', 'FN rate', 'PPV', 'NPV', 'Prevalence']
    df = pd.DataFrame(index=index, columns=['Percent'])
    df.Percent = fpr, fnr, ppv, npv, prev
    df.index.name = name
    return df

Compute metrics for all defendants.

In [35]:
compute_metrics(matrix_all, 'All defendants')

,Percent
All defendants,
FP rate,32.349230
FN rate,37.403876
PPV,61.350618
NPV,68.796510
Prevalence,45.065151


Comparing these results to the table from ProPublica, it looks like our analysis agrees with theirs.

<img src='https://raw.githubusercontent.com/AllenDowney/ElementsOfDataScience/master/machine_bias_table.png'>


We can compute the same metrics for black defendants.

In [36]:
compute_metrics(matrix_black, 'Black defendants')

,Percent
Black defendants,
FP rate,44.846797
FN rate,27.985271
PPV,62.971481
NPV,65.045992
Prevalence,51.433983


And for white defendants.

In [37]:
compute_metrics(matrix_white, 'White defendants')

,Percent
White defendants,
FP rate,23.454301
FN rate,47.722567
PPV,59.133489
NPV,71.187500
Prevalence,39.364303


All of these results are consistent with those reported in the article, including the headline results:

1. The false positive rate for black defendants is substantially higher than for white defendants (45%, compared to 23%).

2. The false negative rate for black defendants is substantially lower (28%, compared to 48%).

In [38]:
error_rates(matrix_black)

(44.84679665738162, 27.985270910047344)

In [39]:
error_rates(matrix_white)

(23.45430107526882, 47.72256728778468)


* Of all people who *will* recidivate, black defendants are more likely than white defendants to be classified as high risk, and (if these classifications influence sentencing decisions) more likely to be sent to prison.  

* Of all people who *will not* recidivate, white defendants are less likely than black defendants to be be classified as high risk and sent to prison.

This seems clearly unfair, but it turns out that "fair" is complicated.  After the ProPublica article, the Washington Post published a response by Sam Corbett-Davies, Emma Pierson, Avi Feller and Sharad Goel: "[A computer program used for bail and sentencing decisions was labeled biased against blacks. It’s actually not that clear.](https://www.washingtonpost.com/news/monkey-cage/wp/2016/10/17/can-an-algorithm-be-racist-our-analysis-is-more-cautious-than-propublicas/)"

I encourage you to read that article, and then run [the next notebook](), where we will unpack their arguments and replicate their analysis.